In [1]:
import tensorflow as tf
import numpy as np

In [2]:
epochs = 10
batch = 256

In [3]:
def build_cnn():
    inputs = tf.keras.Input(shape=(28,28,1))
    x = tf.keras.layers.Conv2D(8,(3,3),padding='same',activation='relu')(inputs)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(10,activation='softmax')(x)
    model = tf.keras.Model(inputs,outputs)
    return model

## Using numpy array as inputs

In [4]:
(X_train_1,y_train_1),(X_test_1,y_test_1) = tf.keras.datasets.mnist.load_data()
X_train_1 = X_train_1.astype(np.float32)/255.
X_train_1 = np.expand_dims(X_train_1,axis=-1)
X_test_1 = X_test_1.astype(np.float32)/255.
X_test_1 = np.expand_dims(X_test_1,axis=-1)
y_train_1 = tf.keras.utils.to_categorical(y_train_1,10)
y_test_1 = tf.keras.utils.to_categorical(y_test_1,10)

model_1 = build_cnn()
model_1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.fit(X_train_1,y_train_1,epochs=epochs,batch_size=batch,validation_data=(X_test_1,y_test_1),verbose=2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 - 2s - loss: 0.5067 - acc: 0.8754 - val_loss: 0.2356 - val_acc: 0.9361
Epoch 2/10
60000/60000 - 2s - loss: 0.2094 - acc: 0.9414 - val_loss: 0.1704 - val_acc: 0.9527
Epoch 3/10
60000/60000 - 2s - loss: 0.1540 - acc: 0.9575 - val_loss: 0.1364 - val_acc: 0.9624
Epoch 4/10
60000/60000 - 2s - loss: 0.1221 - acc: 0.9659 - val_loss: 0.1110 - val_acc: 0.9704
Epoch 5/10
60000/60000 - 2s - loss: 0.1002 - acc: 0.9721 - val_loss: 0.0964 - val_acc: 0.9728
Epoch 6/10
60000/60000 - 2s - loss: 0.0846 - acc: 0.9767 - val_loss: 0.0845 - val_acc: 0.9756
Epoch 7/10
60000/60000 - 2s - loss: 0.0742 - acc: 0.9795 - val_loss: 0.0839 - val_acc: 0.9755
Epoch 8/10
60000/60000 - 2s - loss: 0.0659 - acc: 0.9812 - val_loss: 0.0778 - val_acc: 0.9775
Epoch 9/10
60000/60000 - 2s - loss: 0.0590 - acc: 0.9836 - val_loss: 0.0728 - va

In [5]:
model_1.evaluate(X_test_1,y_test_1)

10000/10000 [==============================] - 0s 25us/sample - loss: 0.0705 - acc: 0.9782


[0.07052422738065943, 0.9782]

## Using tf.data.Dataset as inputs

In [6]:
def preprocessing(img,label):
    img = tf.cast(img,tf.float64)
    img = img/255.
    img = tf.expand_dims(img,axis=-1)
    label = tf.one_hot(label,10,dtype=tf.int32)
    return img,label

In [7]:
(X_train_2,y_train_2),(X_test_2,y_test_2) = tf.keras.datasets.mnist.load_data()

train_ds = tf.data.Dataset.from_tensor_slices((X_train_2,y_train_2)).map(preprocessing,num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(256).batch(batch).repeat(epochs).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test_2,y_test_2)).map(preprocessing,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(batch).repeat(epochs).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

model_2 = build_cnn()
model_2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_2.fit(train_ds,epochs=epochs,steps_per_epoch=len(X_train_2)//batch,validation_data=test_ds,validation_steps=5,verbose=2)

Epoch 1/10
234/234 - 3s - loss: 0.4867 - acc: 0.8736 - val_loss: 0.2972 - val_acc: 0.9062
Epoch 2/10
234/234 - 3s - loss: 0.2206 - acc: 0.9376 - val_loss: 0.2048 - val_acc: 0.9406
Epoch 3/10
234/234 - 3s - loss: 0.1578 - acc: 0.9561 - val_loss: 0.1610 - val_acc: 0.9578
Epoch 4/10
234/234 - 3s - loss: 0.1216 - acc: 0.9668 - val_loss: 0.1344 - val_acc: 0.9578
Epoch 5/10
234/234 - 3s - loss: 0.0994 - acc: 0.9729 - val_loss: 0.1206 - val_acc: 0.9609
Epoch 6/10
234/234 - 3s - loss: 0.0844 - acc: 0.9769 - val_loss: 0.1083 - val_acc: 0.9633
Epoch 7/10
234/234 - 3s - loss: 0.0748 - acc: 0.9793 - val_loss: 0.1010 - val_acc: 0.9688
Epoch 8/10
234/234 - 3s - loss: 0.0665 - acc: 0.9813 - val_loss: 0.0940 - val_acc: 0.9711
Epoch 9/10
234/234 - 3s - loss: 0.0606 - acc: 0.9832 - val_loss: 0.0909 - val_acc: 0.9742
Epoch 10/10
234/234 - 3s - loss: 0.0547 - acc: 0.9847 - val_loss: 0.0892 - val_acc: 0.9750


In [8]:
model_2.evaluate(test_ds)

400/400 [==============================] - 3s 8ms/step - loss: 0.0688 - acc: 0.9783


[0.06881496772402898, 0.9783]

## tensorflow distributed training with Keras

In [9]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16748366450226833428
]


In [10]:
(X_train_3,y_train_3),(X_test_3,y_test_3) = tf.keras.datasets.mnist.load_data()

mirrored = tf.distribute.MirroredStrategy()
print('\nnumber of replicas in sync: {}'.format(mirrored.num_replicas_in_sync))

batch_per_replica = batch
global_batch = batch_per_replica * mirrored.num_replicas_in_sync
print('global batch: {}'.format(global_batch))


number of replicas in sync: 1
global batch: 256


In [11]:
train_ds_dist = tf.data.Dataset.from_tensor_slices((X_train_3,y_train_3)).map(preprocessing,num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(256).batch(global_batch).repeat(epochs).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_ds_dist = tf.data.Dataset.from_tensor_slices((X_test_3,y_test_3)).map(preprocessing,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(global_batch).repeat(epochs).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

with mirrored.scope():
    model_3 = build_cnn()
    model_3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_3.fit(train_ds_dist,epochs=epochs,steps_per_epoch=len(X_train_3)//batch,validation_data=test_ds_dist,validation_steps=5,verbose=2)

INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CP

In [12]:
model_3.evaluate(test_ds_dist)

400/400 [==============================] - 3s 8ms/step - loss: 0.0690 - acc: 0.9781


[0.0689774940838106, 0.9781]